In [13]:
# ------------------------------------------------------------
# 0.  Imports & config – nothing here should clash with yours
# ------------------------------------------------------------
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
K       = 3                # demos per query
NUM_SAMPLES = 5             # model completions per query
TEMPERATURE = 0.7           # keep same as training loop

# ------------------------------------------------------------
# 1.  Initialise encoder **in eval mode** (weights frozen)
# ------------------------------------------------------------
encoder = MathBERTEncoder(device=DEVICE, trainable=False)
encoder.eval()                                   # no grads!

icl_model = OpenAIICLModel(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)

# ------------------------------------------------------------
# 2.  Load/define the inference set
#     If you already saved a slice elsewhere, just load it.
# ------------------------------------------------------------
gsm8k_data = load_dataset("gsm8k", "main")["train"].select(range(256))  # slice first 200 examples
gsm8k_to_infer = load_dataset("gsm8k", "main")["train"] \
                     .select(range(5))          # ⬅ change as needed

# ------------------------------------------------------------
# 3.  Evaluation loop
# ------------------------------------------------------------
total, correct = 0, 0
all_losses     = []          # optional – to compare prompt quality

for idx in tqdm(range(len(gsm8k_to_infer)), desc="Baseline eval"):
    item       = gsm8k_to_infer[idx]
    Q_inf, A_gt = item["question"], item["answer"]

    # Build demo pool (everything except current query)
    demos = [(d["question"], d["answer"])
             for j, d in enumerate(gsm8k_data) if j != idx]

    # Encode query + candidate demos
    with torch.no_grad():
        q_emb     = encoder.encode([Q_inf], detach=True).squeeze(0)
        demo_embs = encoder.encode([q for (q, _) in demos], detach=True)

    # ------- ORIGINAL cosine retrieval -------------------------
    top_k, _ = rc.retrieve_top_k_cosine(
        q_emb, demo_embs, k=min(K, len(demos))
    )
    print(f"🔍 Top-K Indices: {top_k}")
    selected_demos = [demos[i] for i in top_k]          # length = 2

    # ------- Run the ICL model --------------------------------
    responses_nested = sample_responses_per_demo(
        demo_tuples = selected_demos,
        Q_inf       = Q_inf,
        icl_model   = icl_model,
        num_samples = NUM_SAMPLES,
        parallel=True
    )
    print(responses_nested)
    flat_responses = [r for demo_resps in responses_nested for r in demo_resps]
    acc = compute_demo_accuracy(flat_responses, A_gt)
    print(acc) 
    correct += acc
    total   += 1

baseline_acc = correct / total
print(f"\n📊  Baseline accuracy on GSM8K_to_infer: {baseline_acc:.3%}")


Baseline eval:   0%|          | 0/5 [00:00<?, ?it/s]

🔍 Top-K Indices: [81, 159, 35]
------------Demo 1--------
Q: Irene earns $500 if she works for 40 hours a week and gets an extra $20 for every hour of overtime. If she worked 50 hours last week, calculate her total income.
A: If Irene worked 50 hours last week, the total number of hours counting as overtime is 50-40 = <<50-40=10>>10 hours.
Since she's given $20 for every hour of overtime, she earned 10*$20 = $<<10*20=200>>200 in overtime.
Her total income, including the overtime, is $500+$200= $<<500+200=700>>700
#### 700

Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A:
144


Baseline eval:  20%|██        | 1/5 [00:04<00:17,  4.50s/it]

[['144', '72', '144', '72', '72'], ['72', '72', '72', '48 + (48/2) = 48 + 24 = 72', '72'], ['48 + 24 = 72', '48 + 24 = 72', '48 + 24 = 72', '48 + 24 = 72', '48 + (48 / 2) = 48 + 24 = 72']]
0.8666666666666667
🔍 Top-K Indices: [110, 147, 81]
------------Demo 1--------
Q: Paul went to a shop to buy some groceries. He bought some bread for $2, butter for $3, and juice for two times the price of the bread. He had $15 for his shopping. How much money did Paul have left?
A: The price of the juice was 2 * 2 = $<<2*2=4>>4.
So in total he paid for his shopping 2 + 3 + 4 = $<<2+3+4=9>>9.
That means he had 15 - 9 = $<<15-9=6>>6 left.
#### 6

Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A:
10


Baseline eval:  40%|████      | 2/5 [00:05<00:08,  2.68s/it]

...

[['50', '25', '50', '25', '50'], ['25', '25', '100 - (50 + 15 + 30) = 100 - 95 = 5', '100 - (50 + 15 + 30) = 100 - 95 = 5', '25'], ['25', '25', '$55', '$35', '$55']]
1.0
🔍 Top-K Indices: [13, 93, 228]
------------Demo 1--------
Q: Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?
A: In one hour, there are 3 sets of 20 minutes.
So, Joy can read 8 x 3 = <<8*3=24>>24 pages in an hour.
It will take her 120/24 = <<120/24=5>>5 hours to read 120 pages.
#### 5

Q: Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?
A:
48


Baseline eval:  80%|████████  | 4/5 [00:09<00:02,  2.05s/it]

[['48', '48', '72', '48', '48'], ['48', '48', '48', '48', '54'], ['54', '54', '48', '36', '72']]
0.0
🔍 Top-K Indices: [93, 13, 113]
------------Demo 1--------
Q: John writes 20 pages a day.  How long will it take him to write 3 books that are 400 pages each?
A: He wants to write 3*400=<<3*400=1200>>1200 pages
So it will take him 1200/20=<<1200/20=60>>60 days
#### 60

Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A:
624


Baseline eval: 100%|██████████| 5/5 [00:11<00:00,  2.24s/it]

[['624', '624', '624', '624', '624'], ['624', '312', '312', '312', '312'], ['312', '312', '312', '312', '312']]
0.4

📊  Baseline accuracy on GSM8K_to_infer: 65.333%


In [ ]:
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

# === Settings ===
API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
K = 32
NUM_SAMPLES_PER_DEMO = 5
LEARNING_RATE =  1e-5
MAX_STEPS = 3
TEMPERATURE = 0.7

# === Init ===
encoder = MathBERTEncoder(device=DEVICE, trainable=True)
encoder.train()

icl_model = OpenAIICLModel(api_key=API_KEY, model_name="gpt-4.1-nano", temperature=TEMPERATURE)
optimizer = torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)

gsm8k_data = load_dataset('gsm8k', 'main')['train']
gsm8k_data = gsm8k_data.select(range(256))  # slice first 200 examples
gsm8k_data_to_infer = gsm8k_data.select(range(0,5))  # slice first 200 examples

# === Training Loop ===
for step in tqdm(range(MAX_STEPS), desc="Training Steps"):
    print(f"\n=== Training Step {step+1} ===")

    for inference_index in tqdm(range(len(gsm8k_data_to_infer)), desc="Examples"):
        inference_item = gsm8k_data_to_infer[inference_index]
        demo_pool = [d for idx, d in enumerate(gsm8k_data) if idx != inference_index]

        Q_inf = inference_item["question"]
        A_gt = inference_item["answer"]
        demos = [(d["question"], d["answer"]) for d in demo_pool]

        q_emb = encoder.encode([Q_inf], detach=False).squeeze(0)
        demo_embs = encoder.encode([q for (q, a) in demos], detach=False)
        
        top_k_indices, _ = rc.retrieve_top_k_cosine(q_emb, demo_embs, k=min(K, len(demos)))
        print("-------Question---------")
        print(Q_inf)
        print(f"\n🧠 Inference Index {inference_index}")
        print(f"🔍 Top-K Indices: {top_k_indices}")
        
        top_k_indices, similarities = rc.retrieve_sample_k_cosine(q_emb, demo_embs, k=min(K, len(demos)))
        selected_demos = [demos[i] for i in top_k_indices]



        all_responses = sample_responses_per_demo(
            demo_tuples=selected_demos,
            Q_inf=Q_inf,
            icl_model=icl_model,
            num_samples=NUM_SAMPLES_PER_DEMO,
            parallel=True
        )

        rewards = []
        for i, responses in enumerate(all_responses):
            reward = compute_demo_accuracy(responses, A_gt)
            rewards.append(reward)
            # print(f"    Demo {i} | Reward: {reward:.2f}")

        rewards = torch.tensor(rewards, dtype=torch.float32).to(DEVICE)

        loss = grpo_step(
            rewards,
            similarities,
            q_emb,
            demo_embs,
            optimizer
        )

        print(f"✅ Loss: {loss:.4f}")



Training Steps:   0%|          | 0/3 [00:00<?, ?it/s]


=== Training Step 1 ===


-------Question---------
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

🧠 Inference Index 0
🔍 Top-K Indices: [176, 50, 159, 81, 163, 109, 84, 247, 203, 52, 51, 201, 202, 161, 89, 105, 56, 250, 104, 254, 110, 4, 86, 144, 95, 59, 0, 29, 8, 41, 2, 167]
------------Demo 1--------
Q: Samantha’s last name has three fewer letters than Bobbie’s last name. If Bobbie took two letters off her last name, she would have a last name twice the length of Jamie’s. Jamie’s full name is Jamie Grey. How many letters are in Samantha’s last name?
A: There are 4 letters in Jamie’s last name, so Bobbie’s name is 4*2 +2 = <<4*2+2=10>>10 letters long.
Samantha’s last name is 3 letters shorter than Bobbie’s, so there are 10 - 3 = <<10-3=7>>7 letters in Samantha’s last name.
#### 7

Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia

In [12]:
# ------------------------------------------------------------
# 0.  Imports & config – nothing here should clash with yours
# ------------------------------------------------------------
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
K       = 3                # demos per query
NUM_SAMPLES = 5             # model completions per query
TEMPERATURE = 0.7           # keep same as training loop

# ------------------------------------------------------------
# 1.  Initialise encoder **in eval mode** (weights frozen)
# ------------------------------------------------------------
encoder.eval()                                   # no grads!

icl_model = OpenAIICLModel(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)

# ------------------------------------------------------------
# 2.  Load/define the inference set
#     If you already saved a slice elsewhere, just load it.
# ------------------------------------------------------------
gsm8k_data = load_dataset("gsm8k", "main")["train"].select(range(256))  # slice first 200 examples
gsm8k_to_infer = load_dataset("gsm8k", "main")["train"] \
                     .select(range(5))          # ⬅ change as needed

# ------------------------------------------------------------
# 3.  Evaluation loop
# ------------------------------------------------------------
total, correct = 0, 0
all_losses     = []          # optional – to compare prompt quality

for idx in tqdm(range(len(gsm8k_to_infer)), desc="Baseline eval"):
    item       = gsm8k_to_infer[idx]
    Q_inf, A_gt = item["question"], item["answer"]

    # Build demo pool (everything except current query)
    demos = [(d["question"], d["answer"])
             for j, d in enumerate(gsm8k_data) if j != idx]

    # Encode query + candidate demos
    with torch.no_grad():
        q_emb     = encoder.encode([Q_inf], detach=True).squeeze(0)
        demo_embs = encoder.encode([q for (q, _) in demos], detach=True)

    # ------- ORIGINAL cosine retrieval -------------------------
    top_k, _ = rc.retrieve_top_k_cosine(
        q_emb, demo_embs, k=min(K, len(demos))
    )
    print(f"🔍 Top-K Indices: {top_k}")
    selected_demos = [demos[i] for i in top_k]          # length = 2

    # ------- Run the ICL model --------------------------------
    responses_nested = sample_responses_per_demo(
        demo_tuples = selected_demos,
        Q_inf       = Q_inf,
        icl_model   = icl_model,
        num_samples = NUM_SAMPLES,
        parallel=True
    )
    print(responses_nested)
    flat_responses = [r for demo_resps in responses_nested for r in demo_resps]
    acc = compute_demo_accuracy(flat_responses, A_gt)
    print(acc) 
    correct += acc
    total   += 1

baseline_acc = correct / total
print(f"\n📊  Baseline accuracy on GSM8K_to_infer: {baseline_acc:.3%}")


Baseline eval:   0%|          | 0/5 [00:00<?, ?it/s]

🔍 Top-K Indices: [49, 138, 8]
------------Demo 1--------
Q: Gerald spends $100 a month on baseball supplies. His season is 4 months long. He wants to use the months he's not playing baseball to save up by raking, shoveling, and mowing lawns. He charges $10 for each. How many chores does he need to average a month to save up for his supplies?
A: He needs to save up $400 because 4 x 100 = <<4*100=400>>400
He has 8 months to earn this money because 12 - 4 = <<12-4=8>>8
He needs to earn $50 a month because 400 / 8 = <<400/8=50>>50
He needs to do 5 tasks a month because 50 / 10 = <<50/10=5>>5
#### 5

Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A:
72


Baseline eval:  20%|██        | 1/5 [00:01<00:06,  1.55s/it]

[['72', '72', '72', '72', '72'], ['48 + 24 = 72', '48 + (48 / 2) = 48 + 24 = 72', '48 + 24 = 72', '48 + 24 = 72', '48 + (48 / 2) = 48 + 24 = 72'], ['72', '72', '72', '72', '144']]
0.9333333333333333
🔍 Top-K Indices: [12, 177, 221]
------------Demo 1--------
Q: Jasper will serve charcuterie at his dinner party. He buys 2 pounds of cheddar cheese for $10, a pound of cream cheese that cost half the price of the cheddar cheese, and a pack of cold cuts that cost twice the price of the cheddar cheese. How much does he spend on the ingredients?
A: A pound of cream cheese cost $10 / 2 = $<<10/2=5>>5.
A pack of cold cuts cost $10 x 2 = $<<10*2=20>>20.
Jasper spent $10 + $5 + $20 = $<<10+5+20=35>>35 on the ingredients.
#### 35

Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A:
10


Baseline eval:  40%|████      | 2/5 [00:03<00:04,  1.50s/it]

...

[['50', '50', '50', '50', '50'], ['55', '25', '25', '25', '70'], ['55', '100 - (50 + 15 + 30) = 100 - 95 = 5', '55', '55', '55']]
0.9333333333333333
🔍 Top-K Indices: [5, 49, 8]
------------Demo 1--------
Q: Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?
A: He eats 32 from the largest pizzas because 2 x 16 = <<2*16=32>>32
He eats 16 from the small pizza because 2 x 8 = <<2*8=16>>16
He eats 48 pieces because 32 + 16 = <<32+16=48>>48
#### 48

Q: Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?
A:
54


Baseline eval:  80%|████████  | 4/5 [00:07<00:02,  2.14s/it]

[['54', '36', '48', '33', '54'], ['48', '48', '48', '48', '48'], ['54', '36', '54', '54', '54']]
0.0
🔍 Top-K Indices: [206, 156, 175]
------------Demo 1--------
Q: John volunteers at a shelter twice a month for 3 hours at a time.  How many hours does he volunteer per year?
A: He volunteers 2*12=<<2*12=24>>24 times a year
So he volunteers for 24*3=<<24*3=72>>72 hours
#### 72

Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A:
624


Baseline eval: 100%|██████████| 5/5 [00:09<00:00,  1.92s/it]

[['624', '312', '624', '312', '624'], ['312', '312', '312', '312', '312'], ['3 pages × 2 friends × 2 times a week × 52 weeks = 3 × 2 × 2 × 52 = 624', '312', '312', '312', '312']]
0.26666666666666666

📊  Baseline accuracy on GSM8K_to_infer: 62.667%


In [ ]:
# Save the updated MathBERT model
save_path = "./updated_mathbert"  # your save directory
encoder.model.save_pretrained(save_path)
encoder.tokenizer.save_pretrained(save_path)

# LOADING

# from transformers import BertTokenizer, BertModel

# model = BertModel.from_pretrained("./updated_mathbert")
# tokenizer = BertTokenizer.from_pretrained("./updated_mathbert")
